In [16]:
# 线性回归
import torch
import torch.utils.data as Data
import torch.nn as nn
import numpy as np

In [6]:
# 数据
num_inputs = 2
num_examples  = 1000
true_w = [2 , -3.4]
true_b = 4.2
features = torch.randn(num_examples , num_inputs)
labels = true_w[0] * features[:,0] + true_w[1] * features[:,1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [13]:
# 包装数据
batch_size = 10
dataset = Data.TensorDataset(features, labels) # 将特征与标签组合
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True) # 随机读取小批量数据

In [39]:
# 定义模型
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    def forward(self, x): # 定义前向传播
        y = self.linear(x)
        return y

In [41]:
net = LinearNet(num_inputs)

In [47]:
[*net.parameters()] # 查看模型参数

[Parameter containing:
 tensor([[ 0.0958, -0.1949]], requires_grad=True),
 Parameter containing:
 tensor([-0.2861], requires_grad=True)]

In [52]:
net.linear.weight

Parameter containing:
tensor([[ 0.0958, -0.1949]], requires_grad=True)

In [53]:
net.linear.bias

Parameter containing:
tensor([-0.2861], requires_grad=True)

In [54]:
# 初始化模型参数
from torch.nn import init
init.normal_(net.linear.weight, mean=0, std=0.01) # 初始化权重
init.constant_(net.linear.bias, val=0) # 初始化偏置

Parameter containing:
tensor([0.], requires_grad=True)

In [56]:
[*net.parameters()]

[Parameter containing:
 tensor([[-0.0112, -0.0117]], requires_grad=True),
 Parameter containing:
 tensor([0.], requires_grad=True)]

In [57]:
# 定义损失函数
loss = nn.MSELoss() # 均方误差作为模型的损失函数

In [58]:
# 定义优化算法
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr = 0.03)

In [59]:
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [63]:
optimizer.param_groups

[{'params': [Parameter containing:
   tensor([[-0.0112, -0.0117]], requires_grad=True),
   Parameter containing:
   tensor([0.], requires_grad=True)],
  'lr': 0.03,
  'momentum': 0,
  'dampening': 0,
  'weight_decay': 0,
  'nesterov': False}]

In [65]:
# 训练模型
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))    

epoch 1, loss: 0.000155
epoch 2, loss: 0.000163
epoch 3, loss: 0.000132


In [70]:
true_w,net.linear.weight

([2, -3.4],
 Parameter containing:
 tensor([[ 2.0005, -3.3999]], requires_grad=True))

In [71]:
true_b,net.linear.bias

(4.2,
 Parameter containing:
 tensor([4.1997], requires_grad=True))